In [21]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data_1 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_2/execute_code.jsonl")
data_2 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_3/execute_code.jsonl")
data_3 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_4/execute_code.jsonl")

data_sft = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/sft-data/gemini2.5/3.5k.json"))

data_evo = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/sft-data/sft_industry_optmath_evo.json"))

for d in data_evo:
    # t = {'prompt': question2prompt(d['en_question']), 'completion': d['en_math_model_coptpy_code']}
    import re

    match = re.search(r"Question:\s*([\s\S]+)$", d['prompt'])
    
    question = match.group(1).strip()
        
    d['question'] = question

In [26]:
data_im1 = data_1 + data_2 

data_im2 = data_sft + data_evo+ data_3

In [32]:
def deduplicate_data(data_list):
    """
    根据 'en_question' 或 'question' 键对列表进行去重
    """
    seen_questions = set()
    unique_data = []
    
    for item in data_list:
        # 优先获取 'en_question'，如果没有则获取 'question'
        # 如果两个键都没有，则跳过或根据需求处理
        q_text = item.get('en_question') or item.get('question')
        d['en_question'] = q_text
        
        if q_text is not None:
            if q_text not in seen_questions:
                unique_data.append(item)
                seen_questions.add(q_text)
        else:
            # 如果数据中完全没有这两个 key，默认保留该条数据或跳过
            # 这里选择保留没有这两个 key 的数据（不参与去重逻辑）
            unique_data.append(item)
            
    return unique_data

# 1. 合并数据
data_im1 = data_1 + data_2
data_im2 = data_sft + data_evo + data_3

# 2. 执行去重
data_im1_unique = deduplicate_data(data_im1)
data_im2_unique = deduplicate_data(data_im2)

# 如果你希望将 data_im1 和 data_im2 合并在一起进行全局去重：
# data_all_unique = deduplicate_data(data_im1 + data_im2)

print(f"data_im1 去重前: {len(data_im1)}, 去重后: {len(data_im1_unique)}")
print(f"data_im2 去重前: {len(data_im2)}, 去重后: {len(data_im2_unique)}")

data_im1 去重前: 2118, 去重后: 1220
data_im2 去重前: 10854, 去重后: 10084


In [33]:
import random 

data_im2_sample = random.sample(data_im2_unique, 4000)

In [34]:
import json

with open('/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_im1.json', 'w', encoding='utf-8') as f:
    json.dump(data_im1_unique, f, ensure_ascii=False)

with open('/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_im2.json', 'w', encoding='utf-8') as f:
    json.dump(data_im2_sample, f, ensure_ascii=False)

In [35]:
import json
import math

# 假设 data_im2_sample 是你的原始列表
total_size = len(data_im2_sample)
chunk_size = math.ceil(total_size / 3)  # 计算每份的大小

base_path = '/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_im2'

for i in range(3):
    start = i * chunk_size
    end = (i + 1) * chunk_size
    chunk = data_im2_sample[start:end]
    
    # 生成文件名，如 data_im2_part1.json
    file_path = f"{base_path}_part{i+1}.json"
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(chunk, f, ensure_ascii=False, indent=4)

print(f"拆分完成，每份约 {len(chunk)} 条数据。")

拆分完成，每份约 1332 条数据。


In [11]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data_1 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/im1_not_selected/execute_code.jsonl")
data_2 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/im2_not_selected/execute_code.jsonl")

In [4]:
for d in data_2:
    # t = {'prompt': question2prompt(d['en_question']), 'completion': d['en_math_model_coptpy_code']}
    import re

    match = re.search(r"Question:\s*([\s\S]+)$", d['prompt'])
    
    question = match.group(1).strip()
        
    d['question'] = question

In [6]:
data_2[10]

{'prompt': "You are tasked with solving an operations research problem through a structured three-step process:  \n        1. Reasoning: Understand and analyze the problem.  \n        2. Modeling: Formulate a precise mathematical model.  \n        3. Implementation: Translate the model into executable Python code using `coptpy`.\n\n        Please respond in the following exact format:\n\n        <thinking>\n        Your step-by-step reasoning and interpretation of the problem here.\n        </thinking>\n        <model>\n        Your precise mathematical model formulation here, including decision variables, objective function, and constraints.\n        </model>\n        <code>\n        Your complete and executable Python code using `coptpy` that implements the above model.\n        </code>\n\n        Question: An environmental organization is monitoring the ecological environment of a wildlife conservation area. The conservation area is divided into 5 different zones, each with differen

In [9]:
def is_numeric_str(s):
    if not isinstance(s, str):
        return False
    try:
        float(s)  # 尝试转成浮点数（兼容整数和小数）
        return True
    except ValueError:
        return False

data_filter = [
    d for d in data_2
    if not is_numeric_str(d.get('execution_best_solution'))
]

In [14]:
data_filter += data_1

In [13]:
for d in data_1:
    d['question'] = d['en_question']

In [15]:
len(data_filter)

4766

In [16]:
with open('/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/sample_4k.json', 'w', encoding='utf-8') as f:
    json.dump(data_filter, f, ensure_ascii=False)

In [28]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data_1 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/eval_results/IndustryOR_fixedV2/execute_code.jsonl")

In [3]:
with open('/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/grpo_final_2.json', 'w', encoding='utf-8') as f:
    json.dump(data_1, f, ensure_ascii=False)

In [32]:
data_1[3]

{'en_question': 'A product consists of three components produced by four workshops, each with a limited number of production hours. Table 1.4 below provides the production rates of the three components. The objective is to determine the number of hours each workshop should allocate to each component to maximize the number of completed products. Formulate this problem as a linear programming problem.\n\nTable 1.4\n\n| Workshop | Production Capacity (hours) | Production Rate (units/hour) |   |   |\n| :------: | :-------------------------: | :--------------------------: | - | - |\n|          |                             | Component 1 | Component 2  | Component 3 |\n|    A     |           100               |      10      |      15     |      5      |\n|    B     |           150               |      15      |      10     |      5      |\n|    C     |           80                |      20      |      5      |      10     |\n|    D     |           200               |      10      |      15  

In [41]:
import json
import os

def flatten_retrieval_results(input_path):
    if not os.path.exists(input_path):
        print(f"错误: 找不到文件 {input_path}")
        return

    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    flattened_list = []

    for query_block in data:
        retrieved_list = query_block.get('retrieved', [])
        
        for entry in retrieved_list:
            # 创建一个新字典，先拷贝外层的所有 key (rank, similarity, generated_hint 等)
            new_entry = {k: v for k, v in entry.items() if k != 'item'}
            
            # 获取 item 字典
            item_data = entry.get('item', {})
            
            # 将 item 内部的所有 key-value 直接更新到新字典中 (平铺)
            if isinstance(item_data, dict):
                new_entry.update(item_data)
            
            flattened_list.append(new_entry)

    # 打印结果示例
    print(f"处理完成！")
    print(f"总计整合条目: {len(flattened_list)}")
    if flattened_list:
        print(f"单条数据示例 Keys: {list(flattened_list[0].keys())}")

    # 保存文件
    output_path = '/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/flattened_retrieval_items.json'
    for d in flattened_list:
        d['question'] = d['en_question']
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(flattened_list, f, ensure_ascii=False, indent=2)
    
    print(f"整合后的平铺数据已保存至: {output_path}")

# 执行脚本
file_path = "/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/classified_all_retrieval_results_with_hints.json"
flatten_retrieval_results(file_path)

处理完成！
总计整合条目: 2180
单条数据示例 Keys: ['rank', 'corpus_index', 'similarity', 'math_class_match', 'generated_hint', 'en_question', 'answer', 'dataset_source', 'math_class', 'business_type', 'origin_file']
整合后的平铺数据已保存至: /home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/flattened_retrieval_items.json


In [51]:
import json

data_1 = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/sft-data/sft_filtered.json"))

In [52]:
len(data_1)

7672

In [53]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data_1 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/train_all.jsonl")

In [55]:
for d in data_1:
    d['en_question'] = d['question']

In [ ]:
data=  load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/eval_results/NL4OPT/execute_code.jsonl")

data_wrong = []
for d in data:
    if d['en_answer'] != d['execution_best_solution']:
        data_wrong.append(d)

len(data_wrong)

In [76]:
data_wrong = []
for d in data:
    if d['en_answer'] != d['execution_best_solution']:
        data_wrong.append(d)

len(data_wrong)

61

In [77]:
61 / len(data)

0.24897959183673468

In [59]:
def judge_answer(pred_answer, gt_answer, tolerance=1e-5):

    if pred_answer is None:
        return False
    
    # 转换为字符串进行统一处理
    pred_str = str(pred_answer).strip()
    gt_str = str(gt_answer).strip()
    
    # 情况2: GT 是 "No Best Solution" → 必须完全匹配
    if gt_str == "No Best Solution":
        return pred_str == "No Best Solution"
    
    # 情况3: pred 是 "No Best Solution" 但 GT 不是 → 错误
    if pred_str == "No Best Solution":
        return False
    
    # 情况4: 数值比较
    try:
        pred_float = float(pred_str)
        gt_float = float(gt_str)
        
        if gt_float == 0:
            # GT 为 0 时，使用绝对误差
            return abs(pred_float) <= tolerance
        else:
            # GT 非 0 时，使用相对误差
            relative_err = abs((pred_float - gt_float) / gt_float)
            return relative_err <= tolerance
    except (ValueError, TypeError):
        # 无法转换为数值 → 错误
        return False

judge_answer(2333.33, 2333.3333333333335)

True

NLOPT

生产批量为整数
排班
生产
“批次”、“班次”、“人”、“瓶”
牛奶
药片
桶
面条
不可分割物
餐
瓶
整批
苹果
总量约束
服务员

In [3]:
# IndustryOR_fixedV2
data=  load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/eval_results/IndustryOR_fixedV2/execute_code.jsonl")

data_wrong = []
for d in data:
    if not judge_answer(d['en_answer'], d['execution_best_solution']):
        data_wrong.append(d)

len(data_wrong)

484

设施选址与指派

它最显著的特征是：容器的数量本身是一个变量，我们需要决定“开启”多少个容器，并同时决定每个容器里装什么。

大M法，如果就

它最显著的特征是：容器的数量本身是一个变量，我们需要决定“开启”多少个容器，并同时决定每个容器里装什么。

“离散型资源配置与多工序流量平衡问题”

看到 "Either type" 或 "Total capacity" 时，约束应写成 $x + y \le N$，而非各自限制

漏掉了 “工序平衡约束”（即：A阶段产出的总量必须等于B阶段投入的总量）。
如果出现 .333/.666 $\rightarrow$ 检查是否漏了 INTEGER。结果为 Infeasible (无解) $\rightarrow$ 检查是否漏了“开关变量”，导致所有箱子/容器都被强制开启。

只要看到“如果……就……”（If... then...），立刻引入一个二进制变量 $y \in \{0, 1\}$。大 M 法通用公式：启用限制：$x \le M \cdot y$ （如果不启用，数量 $x$ 强制为 0）。起步量限制：$x \ge \text{Min} \cdot y$ （如果启用，数量 $x$ 强制大于起步量）。

单位要对


以后遇到**目标规划（Goal Planning）**题目：

不要把软目标写成 ==：除非题目明确说“必须”。

检查资源供需：如果总需求大于特定偏好的总供给，强制相等必然无解。

多优先级处理：给优先级高的偏差变量赋予一个极大的系数（如 1e6）。

以后遇到“包、袋、箱”这种词，请毫不犹豫地把变量设为 INTEGER。
多目标规划注意 


在现实生活中，我们很少只有一个目标。比如你经营一家工厂，你可能既想利润最大化，又想员工加班最少，还想机器闲置率最低。这三个目标往往是冲突的（要利润多，通常就要多加班）。

目标规划的目标函数总是 Minimize $Z = f(d^+, d^-)$。具体放哪个 $d$，取决于你的诉求：不少于目标值：最小化负偏差 $\min d^-$（缺了才罚，多了不罚）。不超过目标值：最小化正偏差 $\min d^+$（多了才罚，缺了不罚）。恰好达到目标值：最小化绝对偏差 $\min (d^- + d^+)$（多了少了都要罚）。

这是本题最关键的地方。题目给出了 $p_1, p_2, p_3$ 三个优先级，这属于多目标规划，而不是简单的线性规划：$p_1$ (第一优先级)：总利润 $\ge 3000$。$p_2$ (第二优先级)：$x_1 \ge 5$。$p_3$ (最终优化目标)：最小化加权闲置时间（Idle Time）。

这个问题非常经典，它揭示了在复杂排班问题（Staffing & Scheduling）
题型识别：混合问题（Blending Problem）

这是运筹学中极其经典的 混合问题。这类问题的特征是：多种原材料（A, B, C）混合成 多种产品（品牌 A, B, C）。产品对原材料的成分有 百分比要求（如 $A \ge 60\%$）。原材料有 库存限制，产品有 加工费 和 售价

TSP问题 有无加入MTZ约束

这个环境监测站的问题属于运筹学中的指派问题（Assignment Problem）的一种变体，更具体地说，它是一个带有资源约束的广义指派问题（Generalized Assignment Problem, GAP）


其核心特征是：你需要决定如何将“任务”（5个区域）分配给“资源”（3个团队），且每个分配都伴随着特定的成本和资源消耗（工时）

1. 逻辑陷阱：“全有或全无”约束（All-or-Nothing）

*下料问题（Cutting Stock Problem）


In [4]:
data_wrong[35]

{'en_question': 'An Italian transportation company needs to move some empty containers from its 6 warehouses (located in Verona, Perugia, Rome, Pescara, Taranto, and Lamezia) to major national ports (Genoa, Venice, Ancona, Naples, Bari). The container inventory at the warehouses is as follows:\n\n|  | Empty Containers |\n|:---:|:---:|\n| Verona | 10 |\n| Perugia | 12 |\n| Rome | 20 |\n| Pescara | 24 |\n| Taranto | 18 |\n| Lamezia | 40 |\n\nThe demand at the ports is as follows:\n\n|  | Container Demand |\n|:---:|:---:|\n| Genoa | 20 |\n| Venice | 15 |\n| Ancona | 25 |\n| Naples | 33 |\n| Bari | 21 |\n\nThe transport is carried out by a fleet of trucks. The cost to transport each container is proportional to the distance traveled by the trucks, with a rate of 30 euros per kilometer. Each truck can carry up to 2 containers. The distances are as follows:\n\n|  | Genoa | Venice | Ancona | Naples | Bari |\n|:---:|:---:|:---:|:---:|:---:|:---:|\n| Verona | $290 \\mathrm{~km}$ | $115 \\mathrm

In [67]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]
# IndustryOR_fixedV2
data=  load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/eval_results/NL4OPT/execute_code_4.jsonl")

data_wrong = []
for d in data:
    if not judge_answer(d['en_answer'], d['execution_best_solution']):
        data_wrong.append(d)

len(data_wrong)

32

In [93]:
data_wrong[28]

{'en_question': 'A biotechnology company has 35,000 units of antibiotics available which is important to the process of making a first-dose and second-dose of vaccines. The first-dose vaccine requires 30 units of antibiotics and 20 mg of gelatine whereas the second-dose vaccine requires 65 units of antibiotics and 60 mg of gelatine. Since the first-dose vaccine is required before the second-dose vaccine, there must be more first-dose than second-dose vaccines manufactured. However, at least 40 second-dose vaccines must be made. How many of each vaccine should be made to minimize the amount of gelatine used?',
 'en_answer': '3200.0',
 'generated_code': '<thinking>\nWe are given a production optimization problem involving two types of vaccines: first-dose and second-dose. The goal is to minimize the total amount of gelatine used, subject to constraints on antibiotic availability, production order, and minimum production requirements.\n\nStep 1: Identify decision variables\n- Let x be the

In [ ]:
print(data[3]['en_question'])

Let's imagine you are a student on a budget, trying to create a meal plan to meet your nutritional needs while keeping your finances in check. You have five different food items to consider: Lentils, Rice, Chicken, Beef, and Tofu. Each of these foods provides a varying amount of protein, carbs, and calories, and they also come with different price tags.

Here is the nutritional content and cost of each food:

- Lentils: For 7 dollars, Lentils provide you with 13 grams of protein, 21 grams of carbohydrates, and 284 calories.
- Rice: Providing 2 grams of protein, a substantial 30 grams of carbohydrates, and 55 calories, Rice comes at 6 dollars.
- Chicken: At the same cost as Rice, Chicken offers 7 grams of protein, 13 grams of carbohydrates, and a hefty 276 calories.
- Beef: Priced at 10 dollars, Beef gives you 18 grams of protein, 8 grams of carbohydrates, and 204 calories.
- Tofu: The cheapest option at 2 dollars, Tofu still provides a good amount of nutrients with 12 grams of protein,

In [18]:
data_wrong[3]

{'en_question': "Imagine you're planning your meals for the week and you want to keep everything balanced: enough nutrients for a healthy lifestyle but also manageable costs. You have six food options to consider: Chicken, Rice, Vegetables, Bread, Almonds, and Salmon. Each of these foods has its own nutritional content in terms of proteins, carbs, and calories, and also its own cost.\n\nHere's a summary of what each food offers and its cost:\n\n- Chicken: Offers 16 grams of protein, 8 grams of carbohydrates, and 50 calories for $2.\n- Rice: Provides 11 grams of protein, 24 grams of carbohydrates, and 90 calories for $3.\n- Vegetables: Gives you 1 gram of protein, 5 grams of carbohydrates, and a hefty 164 calories for $3.\n- Bread: Packs in 6 grams of protein, 26 grams of carbohydrates, and 165 calories for $2.\n- Almonds: Offer 2 grams of protein, 11 grams of carbohydrates, and 51 calories for $4.\n- Salmon: Packs in a big punch with 19 grams of protein, 20 grams of carbohydrates, and 

In [17]:
print(data_wrong[3]['en_question'])

Imagine you're planning your meals for the week and you want to keep everything balanced: enough nutrients for a healthy lifestyle but also manageable costs. You have six food options to consider: Chicken, Rice, Vegetables, Bread, Almonds, and Salmon. Each of these foods has its own nutritional content in terms of proteins, carbs, and calories, and also its own cost.

Here's a summary of what each food offers and its cost:

- Chicken: Offers 16 grams of protein, 8 grams of carbohydrates, and 50 calories for $2.
- Rice: Provides 11 grams of protein, 24 grams of carbohydrates, and 90 calories for $3.
- Vegetables: Gives you 1 gram of protein, 5 grams of carbohydrates, and a hefty 164 calories for $3.
- Bread: Packs in 6 grams of protein, 26 grams of carbohydrates, and 165 calories for $2.
- Almonds: Offer 2 grams of protein, 11 grams of carbohydrates, and 51 calories for $4.
- Salmon: Packs in a big punch with 19 grams of protein, 20 grams of carbohydrates, and provides 238 calories for 

cp.sum() 替换为 sum() 或 cp.quicksum()

In [213]:
def check_answer(calc_val_str, target_val_str):
    try:
        # 尝试把两个字符串都转成浮点数
        diff = abs(float(calc_val_str) - float(target_val_str))
        return diff < 30
    except (ValueError, TypeError):
        # 如果转不动（比如是 "No Best Solution" 或 None），就走到这里
        return False

In [214]:
count = 0
for d in data_wrong:
    if check_answer(d['execution_best_solution'], d['en_answer']):
        count += 1

(count + len(data) - len(data_wrong)) / len(data)

0.5665024630541872

TSP问题，TSP，子回路问题，MTZ约束，平衡，最大流，相等， coptpy.quicksum

最大流问题要相等，而且呢，要使用coptpy.quicksum

最大流和整数是最重要的

标准的TSP问题要注意什么


旅行商问题 (TSP)：子回路消除 (SEC)现象：算出结果远小于标准答案（比如 165 vs 237）。死穴：只写了“入度=1，出度=1”。求解器会走捷径，把路径断成几个互不相连的小圈。解药：必须加 MTZ 约束。引入次序变量 $u_i \in [1, n]$。增加约束：u[i] - u[j] + n * x[i, j] <= n - 1 (其中 $j \neq \text{起点}$)。

最大流问题 (Max Flow)：源汇点特权
现象：算出结果是 0 或某个直连管道的容量（比如 15 vs 72）。

死穴：

目标函数只定义为某一根管子。

对“源点”和“汇点”也加了“流入=流出”的平衡约束。

解药：

目标函数：最大化“流出源点的总和”或“流入汇点的总和”。

守恒约束：只给中间节点加 Inflow == Outflow。


混合整数问题 (MIP)：单位与类型
现象：算出结果有零点几的小数，且总价略低（比如 48.16 vs 49.0）。

死穴：

现实中不可分割的物品（如一盒食物、一台机器）设成了 CONTINUOUS。

目标函数的系数单位（如 ¥/kg）与变量单位（如 克）不匹配。

解药：

定义变量：明确使用 vtype=COPT.INTEGER。

量纲分析：在写 setObjective 前，手动检查 系数 * 变量 的单位是否统一。



In [220]:
# IndustryOR_fixedV2
data=  load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/eval_results/OptMATH_Bench_166/execute_code.jsonl")

data_wrong = []
for d in data:
    if not judge_answer(d['en_answer'], d['execution_best_solution']):
        data_wrong.append(d)

len(data_wrong)

141

In [221]:
count = 0
for d in data_wrong:
    if check_answer(d['execution_best_solution'], d['en_answer']):
        count += 1

(count + len(data) - len(data_wrong)) / len(data)

0.25301204819277107

In [227]:
data_wrong[6]

{'en_question': 'Select two facilities from five potential locations (Node 0, Node 1, Node 2, Node 3, and Node 4) to maximize the minimum distance between any two selected facilities. Each location can either be selected (1) or not selected (0). For every pair of locations, determine whether both are selected as facilities, represented by a binary decision variable. The distances between the locations are scaled by a factor of 1,000,000 and are as follows: Node 0 and Node 1: 19 units, Node 0 and Node 2: 21 units, Node 0 and Node 3: 31 units, Node 0 and Node 4: 35 units, Node 1 and Node 0: 29 units, Node 1 and Node 2: 13 units, Node 1 and Node 3: 46 units, Node 1 and Node 4: 37 units, Node 2 and Node 0: 50 units, Node 2 and Node 1: 22 units, Node 2 and Node 3: 49 units, Node 2 and Node 4: 33 units, Node 3 and Node 0: 18 units, Node 3 and Node 1: 20 units, Node 3 and Node 2: 46 units, Node 3 and Node 4: 33 units, Node 4 and Node 0: 18 units, Node 4 and Node 1: 26 units, Node 4 and Node 2

In [56]:
with open('/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/train_all.json', 'w', encoding='utf-8') as f:
    json.dump(data_1, f, ensure_ascii=False)

In [ ]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data_1 = load_jsonl("/home/work/mllm_datas/yilin/code/OR-dataset/dataset/dpo/critic_output/critic_data.jsonl")

In [13]:
print(data_1[3005]['critic'])

The incorrect solution is missing the critical linking constraints that connect the binary office opening variables (x[i]) with the continuous assignment variables (y[i][j]). Without these constraints, the model allows employees to be assigned to offices that are not opened (when x[i] = 0), which violates the problem's requirement that setup costs are only incurred when an office is actually used. The correct solution includes 'Big-M' constraints of the form x[i][j] ≤ M * y[i] for all i,j, which ensure that if y[i] = 0 (office not opened), then no employees can be assigned to that office (all x[i][j] = 0). The incorrect solution only has demand and capacity constraints, making it possible to assign employees to offices without paying the setup cost, leading to an infeasible or incorrect solution. Additionally, the variable naming is swapped between the two solutions (x vs y for binary/continuous), but the fundamental issue is the missing linking constraints.


In [10]:
import json

data = json.load(open("/home/work/mllm_datas/yilin/code/OR-dataset/dataset/dpo/dpo_ori.json"))

In [25]:
len(data)

7573

In [26]:
data[0]

{'en_question': 'A manufacturing company produces three types of electronic components: A, B, and C. The company needs to decide the number of hours to operate each of its three production lines to maximize profit. Each hour of operation on production line 1 yields a profit of $100 for component A, $150 for component B, and $200 for component C. Each hour of operation on production line 2 yields a profit of $120 for component A, $180 for component B, and $220 for component C. Each hour of operation on production line 3 yields a profit of $140 for component A, $200 for component B, and $240 for component C. The total operating hours for all production lines cannot exceed 100 hours per week, and each production line can operate for a maximum of 40 hours per week. The demand for component A requires at least 2000 units to be produced, the demand for component B requires at least 3000 units to be produced, and the demand for component C requires at least 4000 units to be produced. Please h

In [28]:
questions = set()
data_filter = []
for d in data:
    if d['en_question'] in questions:
        continue
    data_filter.append(d)
    questions.add(d['en_question'])

In [30]:
len(data_filter)

6701

In [31]:
len(data)

7573

In [32]:
data_filter[0]

{'en_question': 'A manufacturing company produces three types of electronic components: A, B, and C. The company needs to decide the number of hours to operate each of its three production lines to maximize profit. Each hour of operation on production line 1 yields a profit of $100 for component A, $150 for component B, and $200 for component C. Each hour of operation on production line 2 yields a profit of $120 for component A, $180 for component B, and $220 for component C. Each hour of operation on production line 3 yields a profit of $140 for component A, $200 for component B, and $240 for component C. The total operating hours for all production lines cannot exceed 100 hours per week, and each production line can operate for a maximum of 40 hours per week. The demand for component A requires at least 2000 units to be produced, the demand for component B requires at least 3000 units to be produced, and the demand for component C requires at least 4000 units to be produced. Please h

In [33]:
data_filter_final = []

for d in data_filter:
    if d['is_regenerated']:
        data_filter_final.append(d)

In [34]:
len(data_filter_final)

1395

In [46]:
prompt = "You are tasked with solving an operations research problem through a structured three-step process:\n1. Reasoning: Understand and analyze the problem.\n2. Modeling: Formulate a precise mathematical model.\n3. Implementation: Translate the model into executable Python code using `coptpy`.\nPlease respond in the following exact format:\n<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>\n<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>\n<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>\nQuestion: "

In [47]:
print(prompt)

You are tasked with solving an operations research problem through a structured three-step process:
1. Reasoning: Understand and analyze the problem.
2. Modeling: Formulate a precise mathematical model.
3. Implementation: Translate the model into executable Python code using `coptpy`.
Please respond in the following exact format:
<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>
<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>
<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>
Question: 


In [48]:
data_filter_final_save = []

for d in data_filter_final:
    t = {'prompt': prompt + d['en_question'], 'completion': d['chosen'][0]}

    data_filter_final_save.append(t)

In [51]:
with open('/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft.json', 'w', encoding='utf-8') as f:
    json.dump(data_filter_final_save, f, ensure_ascii=False)

In [52]:
import json

data = json.load(open("/home/work/mllm_datas/yilin/code/OR-dataset/output_labels/label_grpo.json"))

In [54]:
for d in data:
    d['variable_questions'] = [d['variable_questions'][0]]

In [55]:
data[0]

{'question': "Minimize the total cost over a 25-week period, which includes setup costs for initiating production, production costs based on the quantity produced, holding costs for carrying inventory, and backlogging costs for unmet demand. Each week, decide whether to initiate production, which incurs a fixed setup cost, and determine the exact quantity to produce, which incurs a variable production cost per unit. Inventory can be carried from one week to the next, incurring a holding cost per unit, and unmet demand can be backlogged to future weeks, incurring a penalty cost per unit. The total cost must account for these factors while meeting weekly customer demand.\n\nThe setup costs vary by week and are as follows: 1,693 for Week 1, 2,253 for Week 2, 1,632 for Week 3, 2,253 for Week 4, 1,862 for Week 5, 1,558 for Week 6, 2,033 for Week 7, 2,030 for Week 8, 2,463 for Week 9, 1,978 for Week 10, 2,095 for Week 11, 2,230 for Week 12, 1,805 for Week 13, 2,348 for Week 14, 2,145 for Wee

In [56]:
with open('/home/work/mllm_datas/yilin/code/OR-dataset/output_labels/label_grpo_1.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

In [57]:
data[0]

{'question': "Minimize the total cost over a 25-week period, which includes setup costs for initiating production, production costs based on the quantity produced, holding costs for carrying inventory, and backlogging costs for unmet demand. Each week, decide whether to initiate production, which incurs a fixed setup cost, and determine the exact quantity to produce, which incurs a variable production cost per unit. Inventory can be carried from one week to the next, incurring a holding cost per unit, and unmet demand can be backlogged to future weeks, incurring a penalty cost per unit. The total cost must account for these factors while meeting weekly customer demand.\n\nThe setup costs vary by week and are as follows: 1,693 for Week 1, 2,253 for Week 2, 1,632 for Week 3, 2,253 for Week 4, 1,862 for Week 5, 1,558 for Week 6, 2,033 for Week 7, 2,030 for Week 8, 2,463 for Week 9, 1,978 for Week 10, 2,095 for Week 11, 2,230 for Week 12, 1,805 for Week 13, 2,348 for Week 14, 2,145 for Wee

In [1]:
import pandas as pd

df = pd.read_parquet("/home/work/mllm_datas/yilin/code/video_caption/output_split/batch_00001.parquet")

In [2]:
df['clip_infos'][200]

array([{'clip_score': None, 'error': None, 'mms_vid': 'br-11110177-6kfkq-lvf251p02dyf77', 'process_error': None, 'recaption': 'In a cozy, softly lit corner of what appears to be a kitchen or pantry, a playful and whimsical scene unfolds centered around a Hello Kitty-themed chocolate-glazed donut-shaped toy. The donut, adorned with white icing drizzles and a cheerful red bow, sits prominently on a white tiled surface, nestled beside a blue plastic bag and other household items. A hand, clad in a textured gray sleeve, gently lifts a pink tag attached to the toy, revealing the playful message: “I LOVE KT. Don’t eat this!” The tag, decorated with Hello Kitty’s iconic bow and cheerful branding for the “Hello Kitty Sweets Cafe,” adds a touch of nostalgia and charm to the moment. As the hand moves the tag, the camera subtly shifts, capturing the toy’s glossy chocolate glaze and the tiny yellow decoration in its center, evoking a sense of delight and care. The surrounding environment, with its

In [3]:
count = 0
for d in df['clip_infos']:
    if d[0]['recaption'] is not None:
        count += 1
count

858

In [7]:
858 * 6 * 2 * 24 * 10

2471040

In [27]:
df['clip_infos'][700]

array([{'clip_score': None, 'error': 'no_video', 'mms_vid': '1ed334dc011f94d2a6599b7550be9475', 'process_error': None, 'recaption': None, 'source_file': 'mp-item_stage4_aes4.5_area0.2_score0.75_all.parquet', 'text': 'A young woman in a red top and light blue jeans stands in a minimalist room, capturing her reflection in a full-length mirror. she accessorizes with a black bracelet, a ring, and a small white bag. the room features a wooden chair with a blue cushion, a white side table with a plant, books, and a decorative item, all under natural light. over time, her attire changes slightly, including a red ribbed polo shirt and a red ribbed top with a v-neckline. the background remains consistent, with a white wall, wooden floor, and a window with sheer curtains.', 'timestamp': "['0:00:00.320000', '0:00:12.679000']", 'videoID_old': '1ed334dc011f94d2a6599b7550be9475_0:00:00.320000_0:00:12.679000'}],
      dtype=object)

In [22]:
import json

def load_jsonl(path):
    return [json.loads(line) for line in open(path, encoding='utf-8') if line.strip()]

data = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/eval_results/MAMO_ComplexLP_fixed/execute_code_4.jsonl")

In [23]:
len(data)

203

In [24]:
from collections import  defaultdict
question2d = defaultdict(list)

for d in data:
    question2d[d['en_question']].append(d)

In [25]:
# 整数规划和非整数完全不同，一个用的是 单纯形法 LP 和 MIP问题
# 整数 用分支定界法

In [26]:
final_list = list(question2d.values())

In [27]:
print(final_list[0][1]['en_question'])

IndexError: list index out of range

In [ ]:
print(final_list[0][1]['generated_code'])

<thinking>
We are given a multi-warehouse resource allocation problem where each warehouse has an excess and a deficit. The goal is to move product between warehouses to meet all demands at minimum total transportation cost. This is a classic transportation problem in operations research.

Step 1: Identify types of warehouses  
- Some warehouses have excess (surplus) product.
- Others have a deficit (need more).  

Step 2: Determine supply and demand for each warehouse  
- For each warehouse, compute:
  - Supply = units available (current quantity)
  - Demand = units needed

Let's tabulate them:

| Warehouse | Current (Supply) | Needs (Demand) | Net Requirement (Supply - Demand) |
|------------|------------------|----------------|----------------------------------|
| 1          | 497              | 161            | 336 (Surplus)                    |
| 2          | 39               | 257            | -218 (Deficit)                   |
| 3          | 416              | 252            | 1

In [ ]:
print(final_list[3][1]['en_question'])

IndexError: list index out of range

这是 mamo complex 35 36单位 37用完形填空 39 可以用llm as judge 40 80数值范围 130 140是流守恒 190 llm judge

29 47 50 建模问题 43 45是流问题

In [57]:
i = 201
[d['execution_best_solution'] for d in final_list[i]], final_list[i][-1] # 14 1

(['46.0'],
 {'en_question': 'Imagine a network of highways that connect six major cities in a country. These highways are designed to facilitate the transportation of goods from a central warehouse (City 0) to a main retail hub (City 5). Each highway has a specific transportation capacity, indicating the maximum number of trucks it can accommodate per hour.\n\nHere is the transportation capacity of each highway connecting the cities:\n\n- From City 0 (Warehouse): Can send goods to City 1 (14 trucks), City 2 (15 trucks), City 3 (13 trucks), and City 5 (4 trucks).\n- From City 1: Can send goods to City 0 (10 trucks), City 2 (7 trucks), City 3 (18 trucks), City 4 (9 trucks), and City 5 (5 trucks).\n- From City 2: Can send goods to City 0 (11 trucks), City 3 (7 trucks), City 4 (13 trucks), and City 5 (6 trucks).\n- From City 3: Can send goods to City 0 (7 trucks), City 1 (11 trucks), City 2 (18 trucks), City 4 (17 trucks), and City 5 (2 trucks).\n- From City 4: Can send goods to City 0 (19

In [43]:
final_list[i][-1] # 14 1

{'en_question': "Imagine you're planning your meals for the day and you want to ensure you're meeting all your nutritional needs without overspending. You have five food options: Chicken, Pasta, Rice, Beans, and Cheese. Each of these foods provides different amounts of protein, carbohydrates, and calories, along with their respective costs.\n\nHere's a breakdown of what each food offers and how much it costs:\n\n- Chicken: Provides 5 grams of protein, 13 grams of carbohydrates, and 121 calories for $7.\n- Pasta: Offers 3 grams of protein, 23 grams of carbohydrates, and 274 calories for $10.\n- Rice: Contains 11 grams of protein, 27 grams of carbohydrates, and 251 calories for $7.\n- Beans: Supplies 10 grams of protein, 27 grams of carbohydrates, and 259 calories for $9.\n- Cheese: Gives you 4 grams of protein, 18 grams of carbohydrates, and 199 calories for $7.\n\nYour goal is to get at least 60 grams of protein, 146 grams of carbohydrates, and 1727 calories from any combination of the

In [58]:
print(final_list[i][-1]['to_run_script'])

import coptpy as cp
from coptpy import COPT

# Create a COPT environment and model
env = cp.Envr()
model = env.createModel("MaxFlow")

# Define the set of nodes (cities)
nodes = [0, 1, 2, 3, 4, 5]

# Define the arcs (directed highways) with their capacities
# Arcs are tuples (from_node, to_node, capacity)
arcs = [
    (0, 1, 14), (0, 2, 15), (0, 3, 13), (0, 5, 4),
    (1, 0, 10), (1, 2, 7), (1, 3, 18), (1, 4, 9), (1, 5, 5),
    (2, 0, 11), (2, 3, 7), (2, 4, 13), (2, 5, 6),
    (3, 0, 7), (3, 1, 11), (3, 2, 18), (3, 4, 17), (3, 5, 2),
    (4, 0, 19), (4, 1, 8), (4, 2, 15), (4, 3, 15), (4, 5, 5),
    (5, 0, 10), (5, 1, 19), (5, 2, 17), (5, 3, 2), (5, 4, 3)
]

# Create a dictionary to hold flow variables for each arc
flow_vars = {}
for i, j, cap in arcs:
    var_name = f"x_{i}_{j}"
    flow_vars[(i, j)] = model.addVar(lb=0.0, ub=cap, vtype=COPT.CONTINUOUS, name=var_name)

# Set the objective function: maximize total flow out of source node 0
obj_expr = 0
for i, j, cap in arcs:
    if i ==

In [ ]:
# 运输问题
# model.addConstr(out_flow == in_flow)最大流问题，加入代码自动judge
# 匹配原始数值 15
# 无法判断的，judge model，触发judge model

In [ ]:
# 开始进行check，流问题如何check呢
# LLM judge
# /home/work/mllm_datas/yilin/code/OR-dataset/dataset/error_data/classified_all_retrieval_results.json
# /home/work/mllm_datas/yilin/code/OR-SR1/datasets/error_data_classified/classified_all_retrieval_results.json

# 请你给我处理这两个文件

# 首先在于这个 复原原来的问题，无监督， This is a MIP问题，粗略监督， 

In [53]:
# optmath中的数据，一些整数约束的数据，一些流，一些TSP问题， simple but easy ，随机在其中加入一个hints，流问题，input_flow = ,子环路约束

# 解决模型的这个 能力

# 深层次 建模问题，是不是做一下DPO就好了？，judge model


In [ ]:
# MIP
# 检索到TSP数据
# IndustryOR
# 首先TSP问题如何解决呢
# 首先加入Hints


In [149]:
# 加入一些Industry OR数据
# 加入一些 Optmath数据

data_industry = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/OR-Instruct-Data-3K_final.json"))

data_optmath = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/processed/gurobi_examples_OR_train_processed.json"))

In [150]:
# 给我check下有没有错误
# 然后再次inference，改写我对这个代码
data_industry[20]

{'prompt': "Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA travel agency is planning to provide transportation services for a large tour group of 400 people. They have two types of vehicles to choose from, buses and minibuses. Buses can accommodate 50 people and cost 2000 yuan, while minibuses can accommodate 25 people and cost 1200 yuan. Due to limitations on vehicle usage and maintenance, the number of buses cannot exceed 10, and the number of minibuses cannot exceed 20. The travel agency's goal is to minimize the total cost while meeting the transportation needs of the entire tour group. How many buses and minibuses should the travel agency choose to minimize the total cost while satisfying all the given conditions?\n\n# Response:",
 'completion': '<think>\nTo solve the optimization problem for the travel agency, we need to build an integer programming mode

In [151]:
len(data_industry)

3000

In [152]:
data_optmath[0]['en_question']

'[{\'content\': \'You are a helpful Assistant with expertise in mathmetical modeling and the Gurobi solver. When the User provides an OR question, you will analyze it, build a detailed mathematical model, and provide the Gurobi code to solve it.\\n\\n    Your response should follow these steps:\\n    1.  <think> \\nCarefully analyze the problem to identify decision variables, objective, and constraints.\\n</think>\\n    2.  <model>Develop a complete mathematical model, explicitly defining:\\n        * Sets\\n        * Parameters\\n        * Decision Variables (and their types)\\n        * Objective Function\\n        * Constraints</model>\\n    3.  <python>Provide the corresponding Gurobi Python code to implement the model.</python>\\n\\n    The output must be in Markdown format, with each step enclosed in the specified tags.\', \'role\': \'system\'}\n {\'content\': "Solve the following mathmetical modeling problem\\nA prominent household goods manufacturer, \'HomeEssentials Co.\', is 

In [144]:
import re
import json

def clean_and_extract_problem(raw_data):
    """
    清洗包含系统提示词和转义字符的复杂结构，只保留核心 OR 问题正文。
    """
    # 1. 如果输入是字符串，先尝试解析为 Python 对象
    if isinstance(raw_data, str):
        try:
            # 处理单引号/双引号混用的不规范 JSON 字符串
            # 这种格式通常是直接从 Python 打印出来的 list[dict]
            import ast
            raw_data = ast.literal_eval(raw_data)
        except:
            return ""

    # 2. 找到角色为 'user' 的 content 内容
    problem_text = ""
    for entry in raw_data:
        if entry.get('role') == 'user':
            problem_text = entry.get('content', "")
            break
    
    if not problem_text:
        return ""

    # 3. 移除常见的前缀提示词（DPO训练通常不需要这些指令）
    patterns_to_remove = [
        r"Solve the following mathematical modeling problem:?\s*",
        r"think step by step\.?\s*",
        r"Your task is to determine.*?\.\s*"
    ]
    
    for pattern in patterns_to_remove:
        problem_text = re.sub(pattern, "", problem_text, flags=re.IGNORECASE | re.DOTALL)

    # 4. 彻底还原转义字符 (\\n -> 换行)
    problem_text = problem_text.encode('utf-8').decode('unicode_escape')
    problem_text = problem_text.replace('\\n', '\n').replace('\\"', '"')

    return problem_text.strip()

# --- 集中处理并覆盖 ---
# 假设你的原始数据在变量 data_optmath 中
for d in data_optmath:
    # 提取并重写原来的 content 字段
    # 也可以直接存为 d['pure_question']
    d['en_question'] = clean_and_extract_problem(d.get('en_question', ""))

# # 保存到新文件
# with open('optmath_cleaned.json', 'w', encoding='utf-8') as f:
#     json.dump(data_optmath, f, ensure_ascii=False, indent=4)

In [145]:
data_optmath[0]

{'en_question': '',
 'answer': '```python\nimport gurobipy as gp\nfrom gurobipy import GRB\n\n# Data\nplants = [\'Plant_A\', \'Plant_B\']\nrdcs = [\'RDC_1\', \'RDC_2\', \'RDC_3\']\nzones = [\'Zone_1\', \'Zone_2\', \'Zone_3\', \'Zone_4\']\n\nplant_capacity = {\n    \'Plant_A\': 8000,\n    \'Plant_B\': 10000\n}\n\nrdc_fixed_cost = {\n    \'RDC_1\': 300000,\n    \'RDC_2\': 250000,\n    \'RDC_3\': 400000\n}\n\nrdc_capacity = {\n    \'RDC_1\': 7000,\n    \'RDC_2\': 9000,\n    \'RDC_3\': 6000\n}\n\nzone_demand = {\n    \'Zone_1\': 4500,\n    \'Zone_2\': 3000,\n    \'Zone_3\': 6000,\n    \'Zone_4\': 3500\n}\n\nplant_to_rdc_cost = {\n    (\'Plant_A\', \'RDC_1\'): 5,\n    (\'Plant_A\', \'RDC_2\'): 7,\n    (\'Plant_A\', \'RDC_3\'): 12,\n    (\'Plant_B\', \'RDC_1\'): 8,\n    (\'Plant_B\', \'RDC_2\'): 4,\n    (\'Plant_B\', \'RDC_3\'): 9\n}\n\nrdc_to_zone_cost = {\n    (\'RDC_1\', \'Zone_1\'): 6,\n    (\'RDC_1\', \'Zone_2\'): 10,\n    (\'RDC_1\', \'Zone_3\'): 15,\n    (\'RDC_1\', \'Zone_4\'): 18,\n

In [153]:
import json

data = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/processed/merged_OR_train_classified.json"))

In [158]:
data[100]

{'prompt': "Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA new media content production team needs to complete three projects, namely Project A, Project B, and Project C. The team consists of three employees: Employee 1, Employee 2, and Employee 3. The time taken by each employee to complete each project and the deadline for each project are shown in the table below:\n\n| Employee\\Project | Project A (days) | Project B (days) | Project C (days) |\n|------------------|-----------------|-----------------|-----------------|\n| Employee 1       | 5               | 7               | 6               |\n| Employee 2       | 6               | 5               | 4               |\n| Employee 3       | 7               | 6               | 5               |\n\n| Project Deadline | Project A (days) | Project B (days) | Project C (days) |\n|------------------|--------------

In [159]:
# Optmath取代表性的800个，然后呢，

data = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/retrieved_4/execute_code.jsonl")

In [174]:
def is_number(s):
    # 如果 s 是 None，直接返回 False，不再尝试转换
    if s is None:
        return False
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

data_with_solution = []
for d in data:
    # 增加一个键是否存在的安全检查
    if 'execution_best_solution' in d and is_number(d['execution_best_solution']):
        data_with_solution.append(d)

In [ ]:
len(data_with_solution) # sample出来1000条


2789

In [178]:
data_classfied = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/processed/merged_OR_train_classified.json"))

In [186]:
data_classfied[1000]

{'prompt': 'Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA bank needs to decide on the allocation of funds across four different business units, namely, the commercial loan department, personal loan department, credit card department, and mortgage loan department. Each business unit has a fixed minimum fund requirement, maximum fund limit, and expected return per unit of funds. The objective of fund allocation is to maximize the total expected return while satisfying the minimum fund requirements and not exceeding the maximum fund limits of all departments.\n\nThe relevant information for each department is as follows:\n\n| Business Unit   | Minimum Fund Requirement (in 万元) | Maximum Fund Limit (in 万元) | Expected Return (in percentage) |\n| -------------- | ------------------------------ | --------------------------- | ------------------------------ |\n| Comme

In [200]:
data_2 = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/error_data_classified/classified_with_hints_qwen.json"))

In [201]:
data_2[0]

{'query_index': 0,
 'query': {'en_question': 'A factory produces two types of food, I and II, and currently has 50 skilled workers. It is known that one skilled worker can produce $10 \\ \\mathrm{kg} / \\ \\mathrm{h}$ of food I or $6 \\ \\mathrm{kg} / \\ \\mathrm{h}$ of food II. According to contract bookings, the weekly demand for these two foods will rise sharply, as shown in Table 1-11. Therefore, the factory has decided to train 50 new workers by the end of the 8th week. It is known that a worker works $40 \\ \\mathrm{h}$ per week, and a skilled worker can train up to three new workers in two weeks (during the training period, both the skilled worker and the trainees do not participate in production). The weekly wage of a skilled worker is 360 yuan, the weekly wage of a trainee during the training period is 120 yuan, and after training, the wage is 240 yuan per week, with the same production efficiency as skilled workers. During the transition period of training, many skilled worke

In [250]:
data_tsp = []

In [251]:
# 先找要sft的
data_1 = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/processed/merged_OR_train_classified.json"))

In [252]:
for d in data_1:
    if 'TSP' in d['problem_type']:
        data_tsp.append(d)

In [253]:
len(data_tsp)

5

In [254]:
data_tsp[0] #

{'en_question': "TechFabric Innovations: Automated Guided Vehicle (AGV) Routing Optimization\n\nTechFabric Innovations operates a state-of-the-art smart factory where Automated Guided Vehicles (AGVs) are critical for efficient material handling. One particular AGV, AGV-007, is responsible for delivering specialized components to various key workstations on the factory floor as part of its daily routine. The AGV must start its route from the Main Warehouse, visit each designated workstation exactly once, and then return to the Main Warehouse to complete its tour. The factory layout and the direct travel distances between these locations are provided below.\n\n**Designated Locations:**\n*   **L1**: Main Warehouse (Starting and Ending Point)\n*   **L2**: Assembly Line 1\n*   **L3**: Quality Control Station\n*   **L4**: Painting Booth\n*   **L5**: Packaging Area\n\n**Direct Travel Distances (in meters) between locations:**\n*   L1 to L2: 30 meters\n*   L1 to L3: 45 meters\n*   L1 to L4: 20

In [257]:
data_2[10]['retrieved_with_hints'][0]

{'rank': 1,
 'corpus_index': 4087,
 'similarity': 0.745900917302592,
 'math_class_match': True,
 'item': {'en_question': 'A wealthy art collector has decided to donate the following items to two museums:\n\n- A Monet painting: $30,000\n- A rare Ming Dynasty vase: $18,000\n- A Stradivarius violin: $50,000\n- A vintage Rolex watch: $25,000\n- Two sapphires: $10,000 each\n- An 18th-century French tapestry: $8,000\n- A pair of Persian rugs: $7,000 each (the collector insists that they must be kept together)\n- A bronze statue from the Renaissance: $12,000\n- A classic Jaguar car: $20,000\n- A set of Georgian silverware: $9,000\n- A unique Fabergé egg: $35,000\n\nThese items must be divided between two museums. How can a mathematical program be formulated and solved using COPTPY to minimize the difference in value between the two allocations?',
  'answer': '## Mathematical Model:\nTo solve the donation division problem for the wealthy art collector, we need to construct a mathematical model

In [258]:
for d in data_2:
    for item in d['retrieved_with_hints']:
        if 'tsp' in item['labels']:
            data_tsp.append(item['item'])

In [261]:
with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_new/tsp_200.json', 'w', encoding='utf-8') as f:
    json.dump(data_tsp, f, ensure_ascii=False)

In [260]:
data_tsp[100]

{'en_question': 'A delivery driver needs to visit six different locations—named A, B, C, D, E, and F—to deliver packages. The driver can start from any location but must visit each location exactly once before returning to the starting point. The primary objective is to minimize the total travel distance. The distances between the locations are given as follows:\n\n- From Location A, it costs 30 units to reach B, 60 units to reach C, 45 units to reach D, 55 units to reach E, and 50 units to reach F.\n- From Location B, it costs 30 units to reach A, 55 units to reach C, 35 units to reach D, 45 units to reach E, and 65 units to reach F.\n- From Location C, it costs 60 units to reach A, 55 units to reach B, 40 units to reach D, 35 units to reach E, and 30 units to reach F.\n- From Location D, it costs 45 units to reach A, 35 units to reach B, 40 units to reach C, 25 units to reach E, and 45 units to reach F.\n- From Location E, it costs 55 units to reach A, 45 units to reach B, 35 units t

In [259]:
len(data_tsp)

204

In [ ]:
data_industry[0] #哪些可以推出来 sample大概300个吧

{'prompt': 'Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA geothermal power plant has three geothermal wells (A, B, and C) with maximum sustainable capacities of A: 1000 units, B: 1500 units, and C: 2000 units. The maximum operating capacity of the power generation equipment is 3000 units. To maintain the pressure in the geothermal field, some of the used hot water or steam needs to be reinjected underground, with a reinjection ratio requirement of 40%. The unit extraction costs for each geothermal well are A: 5 units, B: 4 units, and C: 3 units, and the environmental protection cost is 2 units. Assuming the electricity market demand for a time period t is 2800 units, and each unit of electricity generation brings 1 unit of revenue.\n\nHow should the extraction quantities of the three geothermal wells be scheduled to meet the electricity market demand while ma

In [235]:
data_with_solution[0]

{'en_question': 'A manufacturing company produces three types of products: ProductA, ProductB, and ProductC. The company needs to decide the production quantity of each product, the marketing budget for each product, and the investment in research and development (R&D) to improve product quality and reduce production costs. The production costs and market prices of the products are affected by the R&D investment. The initial production costs and market prices for each product are given in the following Table.\n\n| Product | Initial Production Cost | Market Price |\n|---------|-------------------------|--------------|\n| ProductA | $100 per unit          | $150 per unit|\n| ProductB | $150 per unit          | $200 per unit|\n| ProductC | $200 per unit          | $250 per unit|\n\nFor every $10,000 invested in R&D, the production cost of each product decreases by $5 per unit. The marketing budget increases the sales of each product linearly. The company aims to maximize the total profit 

In [239]:
import numpy as np
np.unique([d['dataset_source'] for d in data_with_solution])

array(['Evo_Step_dataset', 'OR-Instruct-Data-3K',
       'gurobi_examples_OR_train', 'optmath_sample_10k', 'resocratic-29k'],
      dtype='<U24')

In [242]:
data_with_solution_final = []

for d in data_with_solution:
    if d['dataset_source'] == 'gurobi_examples_OR_train' or d['dataset_source'] == 'optmath_sample_10k' or d['dataset_source'] == 'OR-Instruct-Data-3K':
        data_with_solution_final.append(d)

In [ ]:
# 处理这个

len(data_with_solution_final)
# 处理这个

1055

In [279]:
data_tmp = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/tsp_200/execute_code.jsonl")

In [285]:
from collections import Counter

def is_number(s):
    if s is None:
        return False
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

# 第一步：按题目对所有解进行分组
question_groups = {}
for d in data_tmp:
    question = d.get('en_question')
    solution = d.get('execution_best_solution')
    
    if question and is_number(solution):
        if question not in question_groups:
            question_groups[question] = []
        # 存储数值（转为 float 统一格式）和原始字典
        question_groups[question].append((float(solution), d))

# 第二步：对每个题目组进行众数投票
data_with_solution_tmp = []
for question, records in question_groups.items():
    # 提取所有的解数值
    all_solutions = [r[0] for r in records]
    
    # 统计每个数值出现的次数
    counts = Counter(all_solutions)
    
    # 获取出现次数最多的数值 (众数)
    # counts.most_common(1)[0][0] 返回出现频率最高的值
    major_solution_val = counts.most_common(1)[0][0]
    
    # 在该组中找到第一个匹配这个众数数值的原始字典记录
    for val, original_dict in records:
        if val == major_solution_val:
            data_with_solution_tmp.append(original_dict)
            break

print(f"原始数据总数: {len(data_tmp)}")
print(f"众数投票去重后的有效题目数: {len(data_with_solution_tmp)}")

原始数据总数: 816
众数投票去重后的有效题目数: 82


In [293]:
data_industry = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/OR-Instruct-Data-3K/OR-Instruct-Data-3K_final_code_filter.json"))

In [298]:
for d in data_industry:
    prompt_content = d.get('prompt', '')
    q_match = re.search(r'# Question:\s*(.*?)\s*# Response:', prompt_content, re.DOTALL)
    en_question = q_match.group(1).strip() if q_match else ""
    d['en_question'] = en_question
    d['en_math_model_coptpy_code'] = d['completion']

In [300]:
import json

# 定义输出路径
output_path = '/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_new/data_industry.jsonl'

with open(output_path, 'w', encoding='utf-8') as f:
    for entry in data_industry:
        # 将字典转换为 JSON 字符串
        json_record = json.dumps(entry, ensure_ascii=False)
        # 写入文件并换行
        f.write(json_record + '\n')

print(f"数据已成功保存至: {output_path}")

数据已成功保存至: /home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_new/data_industry.jsonl


In [402]:
data_industry_code = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/label_data/data_industry/execute_code.jsonl")

def is_number(s):
    # 如果 s 是 None，直接返回 False，不再尝试转换
    if s is None:
        return False
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

data_industry_code_filter = []
for d in data_industry_code:
    # 增加一个键是否存在的安全检查
    if 'execution_best_solution' in d and is_number(d['execution_best_solution']):
        data_industry_code_filter.append(d)
len(data_industry_code_filter)

1704

In [ ]:
import re
import json
from collections import Counter

def is_number(s):
    if s is None: return False
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

def extract_solution_value(exec_output):
    """
    专门从 exec_output 文本中提取 'Just print the best solution:' 后的数值
    """
    if not exec_output:
        return None
    # 匹配提示词后的数字（支持科学计数法）
    pattern = r"Just print the best solution:\s*([-+]?\d*\.\d+|\d+)"
    match = re.search(pattern, exec_output)
    if match:
        return match.group(1)
    return None

def process_data(data_list):
    question_groups = {}
    
    for d in data_list:
        # 1. 提取干净的 en_question
        prompt_content = d.get('prompt', '')
        q_match = re.search(r'# Question:\s*(.*?)\s*# Response:', prompt_content, re.DOTALL)
        en_question = q_match.group(1).strip() if q_match else ""
        d['en_question'] = en_question
        print(en_question)

        # 2. 从执行结果中提取数值并存入 execution_best_solution
        exec_output = d.get('exec_output', '')
        val = extract_solution_value(exec_output)
        d['execution_best_solution'] = val

        # 3. 按题目分组，用于后续投票
        if en_question and is_number(val):
            if en_question not in question_groups:
                question_groups[en_question] = []
            question_groups[en_question].append((float(val), d))

    # 4. 众数投票选择 Major 方案
    final_results = []
    for question, records in question_groups.items():
        all_vals = [r[0] for r in records]
        # 统计众数
        counts = Counter(all_vals)
        major_val = counts.most_common(1)[0][0]
        
        # 找到第一个符合众数的原始字典并保存
        for val, original_dict in records:
            if val == major_val:
                final_results.append(original_dict)
                break
                
    return final_results

# 示例使用：
process_data(data_industry)

A geothermal power plant has three geothermal wells (A, B, and C) with maximum sustainable capacities of A: 1000 units, B: 1500 units, and C: 2000 units. The maximum operating capacity of the power generation equipment is 3000 units. To maintain the pressure in the geothermal field, some of the used hot water or steam needs to be reinjected underground, with a reinjection ratio requirement of 40%. The unit extraction costs for each geothermal well are A: 5 units, B: 4 units, and C: 3 units, and the environmental protection cost is 2 units. Assuming the electricity market demand for a time period t is 2800 units, and each unit of electricity generation brings 1 unit of revenue.

How should the extraction quantities of the three geothermal wells be scheduled to meet the electricity market demand while maximizing revenue and minimizing costs? Design a scheduling plan and calculate the maximum total revenue.
A travel company offers personalized travel plan services. The company can offer f

[]

In [401]:
len(data_industry)

1932

In [319]:
import random

data_industry_code_filter = random.sample(data_industry_code_filter, 800)

In [320]:
with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/tsp_200.json', 'w', encoding='utf-8') as f:
    json.dump(data_with_solution_tmp, f, ensure_ascii=False)

with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/data_industry.json', 'w', encoding='utf-8') as f:
    json.dump(data_industry_code_filter, f, ensure_ascii=False)

with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/gemini.json', 'w', encoding='utf-8') as f:
    json.dump(data_with_solution_final, f, ensure_ascii=False)

In [308]:
# 分三个模块
data_with_solution_tmp[0]

data_industry_code_filter[0]

{'prompt': 'Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA home services company offers two types of cleaning services: basic cleaning services and deep cleaning services. The company needs to decide how many staff members to allocate to each type of service to meet the demand of at least 50 households.\n\nSpecifically, a household can be served by either one staff member for basic cleaning services or one staff member for deep cleaning services. However, due to considerations of human resources and service quality, the combination of 2 staff members for basic cleaning services and 1 staff member for deep cleaning services cannot exceed 120 households.\n\nThe wage paid to a staff member for basic cleaning services is 7 units, while the wage paid to a staff member for deep cleaning services is 5 units. Due to the nature of the services, both types of staff memb

In [312]:
data_with_solution_final[100]

{'en_question': 'In a molecular analysis scenario, the goal is to assign 4 spectral peaks (Peak 0 to Peak 3) to 5 amino acids (Amino Acid 0 to Amino Acid 4) to minimize the total assignment cost while satisfying specific constraints. Each assignment of a peak to an amino acid has an associated cost, such as assigning Peak 0 to Amino Acid 0 costing 0.613, Peak 0 to Amino Acid 1 costing 0.579, and so on, with all costs explicitly defined. The objective is to minimize the sum of these assignment costs. \n\nThe constraints include ensuring that each amino acid is assigned to at most one peak and each peak is assigned to at most one amino acid. Additionally, exactly 3 assignments must be made in total across all peaks and amino acids. The assignments must also respect Nuclear Overhauser Effect (NOE) constraints, which restrict certain combinations of assignments. For example, if Peak 0 is assigned to Amino Acid 0, then Peak 1 cannot be assigned to Amino Acid 3, and if Peak 0 is assigned to 

In [316]:
len(data_with_solution_final) + 82 + 600

1737

In [ ]:
for d in data_with_solution_tmp:
    d['type'] = 'tsp'

for d in data_industry_code_filter:
    d['type'] = 'industry'

for d in data_with_solution_final:
    d['type'] = 'gemini'

with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/sft_final.json', 'w', encoding='utf-8') as f:
    json.dump(, f, ensure_ascii=False)

In [322]:
len(data_with_solution_tmp + data_industry_code_filter + data_with_solution_final)

1937

In [323]:
data = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/sft_final//execute_code.jsonl")

In [325]:
data[10]

{'prompt': "Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nAn agricultural technology company is developing gene editing technology to improve crop yield and disease resistance. They are developing three types of crop genes: G1, G2, and G3, with research and development costs of $c_{G1}$, $c_{G2}$, and $c_{G3}$, respectively. At the same time, they expect the effects of these three genes to be $e_{G1}$, $e_{G2}$, and $e_{G3}$, respectively.\n\nThe company's goal is to maximize the total effect while minimizing the research and development costs, while satisfying the following constraints:\n\n- The research and development cost for G1 gene cannot exceed $100000.\n- The research and development cost for G2 gene must be at least $50000.\n- The research and development cost for G3 gene cannot exceed $150000.\n- The total research and development cost for all genes c

In [329]:
data_filter = []
for d in data:
    if d['execution_best_solution'] != d['execution_best_solution_new']:
        data_filter.append(d)
len(data_filter)

1057

In [330]:
data_filter[0]

{'prompt': "Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nAn insurance company has 10,000 customers whose policies will expire next month. Based on historical data, the company estimates that each customer has a 0.6 probability of renewing their policy. To encourage policy renewals, the company plans to send a promotional email to all customers next month. However, sending emails incurs a cost of $5 per email. On the other hand, if a customer chooses to renew, the company will earn a profit of $100 per customer.\n\nTo maximize the company's total profit, the company needs to determine the number of emails to send. The company's objective is to send a number of emails that maximizes the expected total profit. The expected total profit can be calculated using the following formula:\n\nExpected Total Profit = Renewal Probability × Number of Customers × Profit per 

In [331]:
with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/sft_final_filter.json', 'w', encoding='utf-8') as f:
    json.dump(data_filter, f, ensure_ascii=False)

In [382]:
data_filter_final = json.load(open("/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/sft_final_filter_regenerated.json"))

In [383]:
print(data_filter_final[800]['en_question'])

Green Building Design Optimization Problem - Numerical Implementation\n\nYou are tasked with optimizing the design of a green building by making specific choices among four distinct design components. The goal is to minimize the total cost while adhering to resource limitations, meeting green certification standards, and complying with technical requirements.\n\n#### Key Decisions:\nYou must decide whether to include or exclude four specific design components, which we will denote as `DesignComponent[0,0]`, `DesignComponent[0,1]`, `DesignComponent[1,0]`, and `DesignComponent[1,1]`. Each decision is binary: `1` if the component is selected, `0` if it is not.\n\n#### Objective:\nMinimize the total overall cost of building design and operation. This total cost is a sum of the combined material, energy, and maintenance costs associated with each selected design component. The individual costs for each component are:\n*   `DesignComponent[0,0]`: 43 units\n*   `DesignComponent[0,1]`: 37 unit

In [369]:
print(data_filter_final[100]['generated_code_new'])

<thinking>
We are given a loyalty program optimization problem involving two types of members: Gold Card and Silver Card.

Step 1: Identify decision variables
- Let G be the total annual mileage flown by Gold Card members.
- Let S be the total annual mileage flown by Silver Card members.

Step 2: Objective function
The goal is to maximize the total mileage rewards for members. Since mileage rewards are based on points earned per mile flown:
- Gold Card: 2 points per mile → reward = 2G
- Silver Card: 1 point per mile → reward = S
Total reward = 2G + S

Step 3: Constraints
1. Flight requirements:
   - Gold Card members must fly at least 5000 miles: G ≥ 5000
   - Silver Card members must fly at least 3000 miles: S ≥ 3000

2. Mileage reward cost constraints:
   - Total mileage rewards for Gold Card members cannot exceed 10000 miles:
     The reward is 2G, which is the total miles earned by Gold members. The problem says "total annual mileage rewards for Gold cannot exceed 10000 miles", whi

In [357]:
data_filter_final[0].keys()

dict_keys(['prompt', 'completion', 'exec_output', 'exec_success', 'en_question', 'execution_best_solution', 'en_math_model_coptpy_code', 'to_run_script', 'execution_result', 'execution_state', 'type', 'generated_code_new', 'execution_best_solution_new', 'en_math_model_coptpy_code_new', 'rewrite_info'])

In [372]:
print(data_filter_final[800]['en_math_model_coptpy_code_new'])

<thinking>
We are given a green building design optimization problem involving four binary decision variables representing whether to include each of four design components. The goal is to minimize total cost subject to resource limits, green certification standards, and technical constraints.

Step-by-step analysis:

1. **Decision Variables:**
   Let `x[i,j]` be a binary variable where `i` is the component index (0 or 1) and `j` is the sub-component index (0 or 1). So we have four variables: `x[0,0]`, `x[0,1]`, `x[1,0]`, `x[1,1]`. These variables are binary (0 or 1).

2. **Objective Function:**
   The objective is to minimize the total cost, which is the sum of the costs of selected components.
   Cost = `43 * x[0,0] + 37 * x[0,1] + 94 * x[1,0] + 88 * x[1,1]`.

3. **Constraints:**

   a. **Resource Constraints:**
      - Resource Type 0: `10 * x[0,0] + 20 * x[1,0] <= 100`
      - Resource Type 1: `15 * x[0,1] + 25 * x[1,1] <= 150`

   b. **Green Certification Constraints:**
      - Gr

In [ ]:
sft_data = json.load(open("/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft.json"))

In [377]:
sft_data[0]

{'prompt': 'You are tasked with solving an operations research problem through a structured three-step process:\n1. Reasoning: Understand and analyze the problem.\n2. Modeling: Formulate a precise mathematical model.\n3. Implementation: Translate the model into executable Python code using `coptpy`.\nPlease respond in the following exact format:\n<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>\n<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>\n<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>\nQuestion: A manufacturer produces three types of electronic components: A, B, and C. The production rate of each component varies depending on the number of skilled workers assigned to each production line. The production rates are as follows: Each worker on line A can produce 10 units of component A per hour, each worker o

In [385]:
data_filter_final[1]

{'prompt': 'Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA school plans to allocate the number of students to four different classes: x1, x2, x3, and x4. The goal is to minimize the total education cost associated with these classes, with education costs per student being $50 for x1, $100 for x2, $200 for x3, and $300 for x4.\n\nThe allocation must comply with the following constraints set due to education resource limitations:\n- The total number of students in classes x1 and x2 must not exceed 500.\n- The total number of students in classes x2 and x3 must be at least 300.\n- The difference in the number of students between classes x3 and x4 must not exceed 100.\n- The difference in the number of students between classes x4 and x1 must be at least 50.\n\nConsidering specific limits on the number of students for each class (x1: 0-500, x2: 0-400, x3: 0-600, x4:

In [386]:
PROMPT = "You are tasked with solving an operations research problem through a structured three-step process:\n1. Reasoning: Understand and analyze the problem.\n2. Modeling: Formulate a precise mathematical model.\n3. Implementation: Translate the model into executable Python code using `coptpy`.\nPlease respond in the following exact format:\n<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>\n<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>\n<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>\nQuestion: "


In [393]:
data_filter_final_sft = []
for d in data_filter_final:
    prompt = PROMPT + "\n" + d['en_question']

    completion = d['en_math_model_coptpy_code_new']
    data_filter_final_sft.append({'prompt': prompt, 'completion': completion})

    
# /home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft.json
with open('/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft_final_filter_1k.json', 'w', encoding='utf-8') as f:
    json.dump(data_filter_final_sft, f, ensure_ascii=False)

In [394]:
with open('/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft_final_2k_0125.json', 'w', encoding='utf-8') as f:
    json.dump(data_filter_final_sft + sft_data, f, ensure_ascii=False)

In [395]:
(data_filter_final_sft + sft_data)[-1]

{'prompt': 'You are tasked with solving an operations research problem through a structured three-step process:\n1. Reasoning: Understand and analyze the problem.\n2. Modeling: Formulate a precise mathematical model.\n3. Implementation: Translate the model into executable Python code using `coptpy`.\nPlease respond in the following exact format:\n<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>\n<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>\n<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>\nQuestion: \\n\n        Imagine a corporate office with five departments, including Recruitment, Marketing, Finance, Research, and IT. They need to share vital financial data amongst departments through their interconnected communication network efficiently. The goal is to determine the maximum amount of financial data (in t

In [397]:
(data_filter_final_sft + sft_data)[0]

{'prompt': 'You are tasked with solving an operations research problem through a structured three-step process:\n1. Reasoning: Understand and analyze the problem.\n2. Modeling: Formulate a precise mathematical model.\n3. Implementation: Translate the model into executable Python code using `coptpy`.\nPlease respond in the following exact format:\n<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>\n<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>\n<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>\nQuestion: \nA city planner is considering allocating resources to four different urban greening projects: x1, x2, x3, and x4. These projects may involve parks, streets, neighborhoods, and schools, respectively. The maintenance costs for each project are 50, 100, 200, and 300 yuan, respectively.\n\nThe allocation must comply

In [398]:
len((data_filter_final_sft + sft_data))

2415

In [399]:
len(sft_data)

1395

In [404]:
data_all = data_filter_final_sft + sft_data

In [405]:
len(data_industry_code_filter)

1704

In [407]:
data_all[0]

{'prompt': 'You are tasked with solving an operations research problem through a structured three-step process:\n1. Reasoning: Understand and analyze the problem.\n2. Modeling: Formulate a precise mathematical model.\n3. Implementation: Translate the model into executable Python code using `coptpy`.\nPlease respond in the following exact format:\n<thinking>Your step-by-step reasoning and interpretation of the problem here.</thinking>\n<model>Your precise mathematical model formulation here, including decision variables, objective function, and constraints.</model>\n<code>Your complete and executable Python code using `coptpy` that implements the above model.</code>\nQuestion: \nA city planner is considering allocating resources to four different urban greening projects: x1, x2, x3, and x4. These projects may involve parks, streets, neighborhoods, and schools, respectively. The maintenance costs for each project are 50, 100, 200, and 300 yuan, respectively.\n\nThe allocation must comply

In [410]:
prompts = set([d['prompt'] for d in data_all])

data_industry_more = []

for d in data_industry_code_filter:
    prompt = PROMPT + "\n" + d['en_question']
    
    if prompt not in prompts:
        data_industry_more.append(d)


In [411]:
len(data_industry_more)

1271

In [412]:
data_industry_more[0]

{'prompt': 'Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA legal consulting company plans to allocate resources between two different services: Artificial Intelligence Legal Consulting Service (Service A) and Traditional Human Legal Consulting Service (Service B). The costs associated with these two services are 20 units per resource for Service A and 30 units per resource for Service B. The goal is to minimize the total cost while satisfying certain constraints imposed by regulations.\n\nDue to legal restrictions, the combined allocation of resources for Service A and Service B cannot exceed 1500 units. Additionally, the allocation for Service A should be at least 500 units more than that for Service B.\n\nUnder these conditions, what is the minimum total cost while meeting the service demands and adhering to the operational constraints? Please round the answ

In [413]:
with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/data_industry_more.json', 'w', encoding='utf-8') as f:
    json.dump(data_industry_more, f, ensure_ascii=False)

In [415]:
# /home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_industry_more/execute_code.jsonl

data_industry_code = load_jsonl("/home/work/mllm_datas/yilin/code/OR-SR1/grpo_data/dataset/data_industry_more/execute_code.jsonl")

def is_number(s):
    # 如果 s 是 None，直接返回 False，不再尝试转换
    if s is None:
        return False
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

data_industry_code_filter = []
for d in data_industry_code:
    # 增加一个键是否存在的安全检查
    if 'execution_best_solution' in d and is_number(d['execution_best_solution']):
        data_industry_code_filter.append(d)
len(data_industry_code_filter)

1271

In [418]:
with open('/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/data_industry_more.json', 'w', encoding='utf-8') as f:
    json.dump(data_industry_code, f, ensure_ascii=False)

In [417]:
data_industry_code[0]

{'prompt': 'Below is an operations research question. Build a mathematical model and corresponding python code using `coptpy` that appropriately addresses the question.\n\n# Question:\nA city has three main public areas that require lighting: a park (P), a pedestrian street (W), and a square (S). The minimum required lighting intensity for each area is P: 1000lm, W: 2000lm, and S: 3000lm. The lighting supply is provided by three different types of lamps: LED lights, energy-saving lights, and incandescent light bulbs, with their lighting intensity and energy consumption per hour as shown in the table below:\n\n| Lamp Type    | Lighting Intensity (lm) | Energy Consumption (kW) |\n| ------------ | ---------------------- | ----------------------- |\n| LED Light    | 500                    | 0.05                    |\n| Energy-Saving Light | 200                    | 0.07                    |\n| Incandescent Light Bulb | 100                    | 0.1                    |\n\nDue to budget and 

In [419]:
import json

tmp = json.load(open("/home/work/mllm_datas/yilin/code/OR-dataset/dataset/sft_final/data_industry_more_regenerated.json"))
tmp_final = []
for d in tmp:
    prompt = PROMPT + "\n" + d['en_question']

    completion = d['en_math_model_coptpy_code_new']
    tmp_final.append({'prompt': prompt, 'completion': completion})

    
# /home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft.json
# with open('/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft_final_filter_1k.json', 'w', encoding='utf-8') as f:
#     json.dump(data_filter_final_sft, f, ensure_ascii=False)
len(tmp_final)

1176

In [422]:
with open('/home/work/mllm_datas/yilin/code/OR-SR1/datasets/trainset/sft_final_3k_0125.json', 'w', encoding='utf-8') as f:
    json.dump(data_filter_final_sft + sft_data + tmp_final, f, ensure_ascii=False)

In [421]:
len(data_filter_final_sft + sft_data + tmp_final)

3591

In [11]:
import pandas as pd

df = pd.read_parquet("/home/work/mllm_datas/yilin/code/video_caption/output_split_test/batch_00007.parquet")

In [12]:
df.head()

,clip_infos,clip_count,videoID,timestamp_cn,captions_interlink7m
0,"[{'clip_score': None, 'mms_vid': 'br-11110177-...",3,0rDsTCF8CWg,2025-12-23 18:19:20.767,[]
1,"[{'clip_score': None, 'mms_vid': 'br-11110177-...",5,0rv9cAXiIRE,2025-12-23 18:19:20.767,[]
2,"[{'clip_score': None, 'mms_vid': 'br-11110177-...",2,0sTY_cpt0D4,2025-12-23 18:19:20.767,[]
3,"[{'clip_score': None, 'mms_vid': 'br-11110177-...",22,0t70303Gp_c,2025-12-23 18:19:20.767,[{'text': 'The video begins with a shot of a t...
4,"[{'clip_score': None, 'mms_vid': 'br-11110177-...",16,0tdwVibYrIA,2025-12-23 18:19:20.767,[]


In [13]:
print(df['clip_infos'][0][0]['recaption'])

In a luxurious, warmly lit display case, a sophisticated and elegant collection of fine artifacts is showcased, centered around a black-framed glass case housing a silver ornate flute-like object labeled “ARTICURIAL” with a smaller plaque reading “LEONARD B. LEEDON” beneath it. The item rests atop a black base, accompanied to its right by a tall, intricate, crystal-encrusted pen or decorative object, possibly a luxury fountain pen or sculptural piece, with a metallic, faceted design that glimmers under the ambient light. To the left, a vibrant coral-like red sculpture, resembling coral or a stylized sea creature, adds a splash of color and contrast against the dark tones. The display sits on a green-tinted glass shelf, hinting at a high-end boutique or museum setting. In the blurred background, a purple handbag—possibly a luxury brand such as Hermès—sits alongside a reflective, circular glass object and a softly glowing wall of warm lights, suggesting a festive or upscale retail enviro

In [14]:
df['clip_infos'][0][1]

{'clip_score': None,
 'mms_vid': 'br-11110177-6kfkp-lvp1wrgefuueb7',
 'process_error': None,
 'recaption': "In an opulent, grandiose jewelry store adorned with chandeliers, marble columns, and ornate gold-trimmed archways, a middle-aged man with silver-gray hair, dressed in a sharp navy blue suit and black tie, stands as the focal point of the video. His hands move expressively as he speaks, gesturing with palms open and then clasping together, suggesting he is explaining or emphasizing a point with earnestness. Behind him, a woman with blonde hair, dressed in a light gray cardigan over a checkered skirt, is seen from the back, viewing jewelry in a glass display case, while other patrons, including a woman in a pink top and a young boy, are seated or browsing in the background. The setting exudes luxury with its warm lighting and rich decor, including illuminated jewelry showcases and elegant red draperies. As the man continues his speech, his facial expressions convey confidence and e

In [15]:
df = pd.read_parquet("/home/work/mllm_datas/yilin/code/video_caption/output_split/batch_00007.parquet")

In [16]:
df['clip_infos'][0][1]

{'clip_score': None,
 'error': None,
 'mms_vid': 'br-11110177-6kfkp-lvp1wrgefuueb7',
 'process_error': None,
 'recaption': 'In an opulent, lavishly decorated jewelry store adorned with gilded columns, ornate chandeliers, and rich red drapery, a man in a sharp navy suit and black tie stands as the focal point of the scene, engaging in what appears to be an earnest conversation or interview. His expressive hand gestures—first spreading apart, then clasping together—suggest he is passionately explaining or emphasizing a point, possibly about the artistry or history of the jewelry displayed around him. Behind him, glass display cases gleam under soft lighting, showcasing sparkling gemstones and intricate pieces that catch the eye of shoppers and staff alike. A woman in a light gray sweater, her back turned to the camera, examines items with quiet interest, while other customers and employees mill about in the softly lit background, creating a sense of bustling yet refined activity. The atm

In [4]:
import pandas as pd

df = pd.read_parquet("/home/work/mllm_datas/yilin/code/video_caption/output_asr/batch_00000.parquet")